In [1]:
# import sys
# {sys.prefix}

{'/opt/homebrew/anaconda3/envs/pipeline'}

In [3]:
# !conda install -c conda-forge -y --prefix {sys.prefix} pandas

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\User\anaconda3\envs\pipeline

  added / updated specs:
    - pandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    numpy-1.22.2               |  py310hcae7c84_0         6.2 MB  conda-forge
    pandas-1.4.1               |  py310hf5e1058_0        11.0 MB  conda-forge
    ------------------------------------------------------------
                                           Total:        17.2 MB

The following NEW packages will be INSTALLED:

  intel-openmp       conda-forge/win-64::intel-openmp-2022.0.0-h57928b3_3663
  libblas            conda-forge/win-64::libblas-3.9.0-13_win64_mkl
  libcblas           conda-forge/win-64::libcblas-3.9.0-13_win64_mkl
  liblapack          conda-forge/win-64::liblapack-3.9.0-13_win64_mkl
  mkl                conda-forge/win-64::mkl-2022.0.0-h0e2418a_796
  numpy

In [32]:
import findspark
findspark.init()

In [33]:
from pyspark.sql import SparkSession
from pyspark.sql.types import ArrayType, IntegerType, StructType, StructField
from pyspark.sql.functions import regexp_extract, lit

In [34]:
from datetime import datetime
import requests
import pandas as pd
import json

In [35]:
ip = '220.78.231.223'
spark_ip = 'spark://'+ip+':7077'
spark_ip

'spark://220.78.231.223:7077'

In [36]:
spark = SparkSession \
        .builder \
        .master('local[2]') \
        .appName('tempera_obs') \
        .getOrCreate()

In [37]:
service_key="1LJG14JWnAylzKi5GvmuGg=="
url="http://www.khoa.go.kr/api/oceangrid/ObsServiceObj/search.do?ServiceKey="+service_key+"&ResultType=json"
url # 디버깅용

'http://www.khoa.go.kr/api/oceangrid/ObsServiceObj/search.do?ServiceKey=1LJG14JWnAylzKi5GvmuGg==&ResultType=json'

In [46]:
now_datetime=datetime.now().strftime('%Y.%m.%d')
now_datetime # 디버깅 용

'2022.02.23'

In [47]:
response = requests.get(url).json()
response['result']['data']

[{'obs_post_id': 'DT_0063',
  'obs_lat': '35.024178',
  'data_type': '조위관측소',
  'obs_post_name': '가덕도',
  'obs_lon': '128.810933',
  'obs_object': '조위,수온,염분,기온,기압,풍속,풍향'},
 {'obs_post_id': 'DT_0032',
  'obs_lat': '37.731944',
  'data_type': '조위관측소',
  'obs_post_name': '강화대교',
  'obs_lon': '126.522222',
  'obs_object': '조위,기압'},
 {'obs_post_id': 'DT_0031',
  'obs_lat': '34.028333',
  'data_type': '조위관측소',
  'obs_post_name': '거문도',
  'obs_lon': '127.308889',
  'obs_object': '조위,수온,염분,기온,기압,풍속,풍향'},
 {'obs_post_id': 'DT_0029',
  'obs_lat': '34.801389',
  'data_type': '조위관측소',
  'obs_post_name': '거제도',
  'obs_lon': '128.699167',
  'obs_object': '조위,수온,염분,기온,기압,풍속,풍향'},
 {'obs_post_id': 'DT_0058',
  'obs_lat': '37.560833',
  'data_type': '조위관측소',
  'obs_post_name': '경인항',
  'obs_lon': '126.601111',
  'obs_object': '조위,기온,기압,풍속,풍향'},
 {'obs_post_id': 'DT_0026',
  'obs_lat': '34.481111',
  'data_type': '조위관측소',
  'obs_post_name': '고흥발포',
  'obs_lon': '127.342778',
  'obs_object': '조위,수온,염분,기온

In [50]:
df = spark.createDataFrame(response["result"]["data"])

In [51]:
df.show(5)

+----------+---------+----------+----------------------------------+-----------+-------------+
| data_type|  obs_lat|   obs_lon|                        obs_object|obs_post_id|obs_post_name|
+----------+---------+----------+----------------------------------+-----------+-------------+
|조위관측소|35.024178|128.810933|조위,수온,염분,기온,기압,풍속,풍향|    DT_0063|       가덕도|
|조위관측소|37.731944|126.522222|                         조위,기압|    DT_0032|     강화대교|
|조위관측소|34.028333|127.308889|조위,수온,염분,기온,기압,풍속,풍향|    DT_0031|       거문도|
|조위관측소|34.801389|128.699167|조위,수온,염분,기온,기압,풍속,풍향|    DT_0029|       거제도|
|조위관측소|37.560833|126.601111|          조위,기온,기압,풍속,풍향|    DT_0058|       경인항|
+----------+---------+----------+----------------------------------+-----------+-------------+
only showing top 5 rows



In [52]:
file_name = "hdfs:///"+ip+':9870/middle_project/'+now_datetime+"_obs_info.csv"
file_name

'hdfs:///220.78.231.223:9870/middle_project/2022.02.23_obs_info.csv'

In [53]:
df.write.csv(file_name)

22/02/23 14:57:57 WARN FileSystem: Failed to initialize fileystem hdfs:///220.78.231.223:9870/middle_project/2022.02.23_obs_info.csv: java.io.IOException: Incomplete HDFS URI, no host: hdfs:///220.78.231.223:9870/middle_project/2022.02.23_obs_info.csv


Py4JJavaError: An error occurred while calling o183.csv.
: java.io.IOException: Incomplete HDFS URI, no host: hdfs:///220.78.231.223:9870/middle_project/2022.02.23_obs_info.csv
	at org.apache.hadoop.hdfs.DistributedFileSystem.initialize(DistributedFileSystem.java:183)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3469)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource.planForWritingFileFormat(DataSource.scala:461)
	at org.apache.spark.sql.execution.datasources.DataSource.planForWriting(DataSource.scala:556)
	at org.apache.spark.sql.DataFrameWriter.saveToV1Source(DataFrameWriter.scala:382)
	at org.apache.spark.sql.DataFrameWriter.

In [8]:
spark.stop()

NameError: name 'spark' is not defined

22/02/23 14:11:47 ERROR TransportResponseHandler: Still have 1 requests outstanding when connection from hadoop01/220.78.231.223:7077 is closed
22/02/23 14:11:47 WARN StandaloneAppClient$ClientEndpoint: Failed to connect to master hadoop01:7077
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:101)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:109)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anon$1.run(StandaloneAppClient.scala:107)
	at java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:511)
	at java.util.concurrent.FutureTask.run(FutureTask.java:266)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecuto